In [42]:
data_consent_info = """DATA CONSENT INFORMATION:

Please read: we wish to record your response data to an anonymised public data repository. Your data will be used for educational teaching purposes
practising data analysis and visualisation. Please type yes in the box below if you consent to the upload."""

print(data_consent_info)

result = input("> ")

if result == "yes":
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")

else:
    # end code execution by raising an exception
    raise(Exception("User did not consent to continue test."))

DATA CONSENT INFORMATION:

Please read: we wish to record your response data to an anonymised public data repository. Your data will be used for educational teaching purposes
practising data analysis and visualisation. Please type yes in the box below if you consent to the upload.


>  yes


Thanks for your participation.
Please contact philip.lewis@ucl.ac.uk
If you have any questions or concerns
regarding the stored results.


In [1]:
id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:

- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise then your unique identifer would be CBTC"""

print(id_instructions)
user_id = input("> ")
print("User entered id:", user_id)
gender_instructions= """

Enter your gender 
Male, Female or Other by typing m,f or o, respectively"""
print(gender_instructions)
user_gender = input ("> ")



Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:

- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise then your unique identifer would be CBTC


>  amd


User entered id: amd


Enter your gender 
Male, Female or Other by typing m,f or o, respectively


>  f


In [2]:
from IPython.display import display, HTML, Image, clear_output
from threading import Timer 
import time
import matplotlib.pyplot as plt
from matplotlib import patches
import json
import numpy as np
import math
import ipywidgets as widgets
from jupyter_ui_poll import ui_events

In [12]:
def no_overlap(circle_center, previous_circles, radius):
    for circle in previous_circles:
        x_dif = circle_center[0] - circle[0]
        y_dif = circle_center[1]- circle[1]
        if math.sqrt(x_dif**2+y_dif**2) < 2*radius:
            return False
    return True      


def circle_in_ellipse(circle_center, ellipse):
    x = circle_center[0]
    y = circle_center[1]
    h, k = ellipse[0]
    a = ellipse[1]
    b = ellipse[2]
    return (x-h)**2/a**2 + (y-k)**2/b**2 <= 1
    
def generate_image(num_left, num_right):
    '''Generate an image with the given number of circles on both sides, and returns the correct answer to the image'''
    #Create the image
    fig, ax = plt.subplots()
    plt.xlim((0,10))
    plt.ylim((0,7.5))
    circle_radius = 0.3

    left_center = (2.5,4)
    left_width = 4
    left_height = 6
    left_ellipse = patches.Ellipse(left_center, left_width, left_height, facecolor = 'none', edgecolor = 'black')
    left_ellipse_data = [left_center, left_width- 10*circle_radius, left_height - 10* circle_radius]

    right_center = (7.5,4)
    right_width = 4
    right_height = 6
    right_ellipse = patches.Ellipse(right_center, right_width, right_height, facecolor = 'none', edgecolor = 'black')
    right_ellipse_data = [right_center, right_width-10*circle_radius, right_height-10*circle_radius]
    ax.add_patch(left_ellipse),
    ax.add_patch(right_ellipse)
    
    previous_left_circles = []
    for left_circle in range(num_left):
        while True:
            circle_x_center = np.random.uniform(left_center[0]-left_width/2  + 0.5, left_center[0]+left_width/2  - 0.5)
            circle_y_center = np.random.uniform(left_center[1]-left_height/2  + 0.5, left_center[1]+left_height/2  - 0.5)
            circle_center = (circle_x_center, circle_y_center)
            if no_overlap(circle_center,previous_left_circles, circle_radius) and circle_in_ellipse(circle_center, left_ellipse_data):
                ax.add_patch(patches.Circle(circle_center, circle_radius, facecolor = 'blue'))
                previous_left_circles.append(circle_center)
                break 
        
    previous_right_circles = []
    for right_circle in range(num_right):
        while True:
            circle_x_center = np.random.uniform(right_center[0]-right_width/2 + 0.5, right_center[0]+right_width/2  - 0.5)
            circle_y_center = np.random.uniform(right_center[1]-right_height/2 + 0.5, right_center[1]+right_height/2 - 0.5)
            circle_center = (circle_x_center, circle_y_center)
            if no_overlap(circle_center,previous_right_circles, circle_radius) and circle_in_ellipse(circle_center, right_ellipse_data):
                ax.add_patch(patches.Circle(circle_center, circle_radius, facecolor = 'orange'))
                previous_right_circles.append(circle_center)
                break
    #fig.patch.set_visible(False)
    ax.axis('off')

    if num_left > num_right:
        answer = 'Left'
    elif num_right > num_left:
        answer = 'Right'
    else:
        answer = 'Error, same number on both sides'
    return answer, fig

circle_numbers = [(1,3), (2,3), (4,3), (5,4), (6,7), (7,5), (8,7), (6,3), (10,9), (11,12), (9,10), (8,9), (9,8), (7,8), (7,3), (7,4), 
                  (8,6), (10,8), (7,10), (11,9), (12,10), (8,5), (6,9), (11, 10), (10, 12)]
data = {}
for i in range(len(circle_numbers)):
    answer, figure = generate_image(circle_numbers[i][0],circle_numbers[i][1])
    file_name = f'circles_figure_{str(i)}.jpg'
    plt.savefig(file_name)
    plt.close()
    data[file_name] = answer

out_file = open("results.json", "w") 
json.dump(data, out_file)
    

In [14]:
import requests
from bs4 import BeautifulSoup
import json

def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}

    loaded_all = True
    for item in result:
        if item[1].strip() not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False

        # Check if the value is an integer, convert to int
        if isinstance(data_dict[item[1]], int):
            form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
        else:
            # Convert other types to strings
            form_dict[f'entry.{item[4][0][0]}'] = str(data_dict[item[1]])

    if loaded_all:
        post_result = requests.post(post_form_url, data=form_dict)
        return post_result.ok
    else:
        return False
  

In [4]:
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

In [8]:
pictures = []
answers = []
with open ("results.json") as my_file: 
    data = json.load(my_file) 
    for pic, answer in data.items():
        pictures.append(Image(pic, width = 300))
        answers.append(answer)

top_area = widgets.Output(layout={"height":"60px"})
main_area = widgets.Output(layout={"height":"300px"})
bottom_area = widgets.Output(layout={"height":"60px"})

btn1 = widgets.Button(description="Left")
btn2 = widgets.Button(description="Right")

btn1.on_click(register_btn_event)
btn2.on_click(register_btn_event)

panel = widgets.HBox([btn1, btn2])

top_area.append_display_data( HTML("<h1>Left or Right</h1>") )
bottom_area.append_display_data(panel)

display(top_area)
display(main_area)
display(bottom_area)

n = len(pictures)
score = 0
np.random.seed(1)
shuffle = np.random.permutation(n)

total_time= 0 

for i in range(n):
    new_i = shuffle[i]
     
    with main_area: display(pictures[new_i])
    start_time= time.time()
    result = wait_for_event(timeout=3)
    end_time= time.time()
    
    if result['description'] == '':
        with main_area: display("Too slow!")
    elif result['description'] ==  answers[new_i]:
        with main_area: display("Correct!")
        score = score + 1
    else:
        with main_area: display("Wrong!")  

    response_time= end_time- start_time
    total_time= response_time+ total_time
        
    wait_for_event(timeout=1.5)
    with main_area: clear_output()
        
with bottom_area: clear_output()
average_time= total_time/ n
with main_area: print(f"You scored {score} and your time was {total_time:.3f} seconds, with an average of {average_time:.3f} seconds per question")

Output(layout=Layout(height='60px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython…

Output(layout=Layout(height='300px'))

Output(layout=Layout(height='60px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chi…

In [17]:
answers_dict = {'id': user_id,
                'gender': user_gender,
                'score': score, 
                'time per test': round(total_time,2),
                'time per question': round(average_time,2)}
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSeIXKlCo7TH-OG_nHCw8TsJ8rD7vJuKFJC-wM0CJAzxszryvQ/viewform"

send_to_google_form(answers_dict, form_url)

True